#### Import required libraries  

In [25]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chat_models import init_chat_model
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from tqdm import tqdm

#### Load PDF file

In [26]:
# Load documents
loader = PyPDFLoader(
    "../docs/courses_offered.pdf",
    mode="page")

docs = loader.load()
print('Loaded document count :', len(docs))

Loaded document count : 15


#### Split document into chunks of 500 chars

In [27]:
# Split documents into chunks

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

chunks = text_splitter.split_documents(docs)
print("Number of chunks created: ", len(chunks))

Number of chunks created:  34


#### Get Google Embedding Model

In [28]:
# Get embeddings model
embeddings_model = GoogleGenerativeAIEmbeddings(
    model="gemini-embedding-001")

#### Create Vector DB

In [29]:
db = FAISS.from_documents(chunks, embeddings_model)
print('Created FAISS index')

Created FAISS index


#### Retrieve Matching Documents

In [30]:
question = "What is the course fee of Python programming course"
#question = "In which course RAG is covered"
retriever = db.as_retriever()
results = retriever.invoke(question)
print("Number of documents retrieved: ", len(results))

Number of documents retrieved:  4


#### Create Context with matching documents

In [31]:
# Create context
context = "\n".join([doc.page_content for doc in results])
#print(context)

#### Create prompt with query and context

In [32]:
prompt = f"""
Give me answer to my question based on the context.
Context : {context}
Question: {question}
"""

In [33]:
#print(prompt)

#### Send Prompt to LLM and get response

In [34]:
# Access LLM with context and query
llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

result = llm.invoke(prompt)
print(result.content)

The course fee for the Python Programming course is Rs. 6000/- (Includes digital course material).
